## **Parsing Model Output**

In [66]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.output_parsers.list import ListOutputParser

In [67]:
llm = OllamaLLM(model="llama3.1:latest")

In [68]:
prompt = PromptTemplate(
    template="List 3 {things}",
    input_variables=["things"]
)

In [69]:
print(llm.invoke(input=prompt.format(things="countries that play cricket in world cup")))

Here are 3 countries that play cricket in the World Cup:

1. India
2. Australia
3. England


In [70]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

In [71]:
print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [72]:
prompt = PromptTemplate(
    template="List 3 {things}.\n{format_instructions}",
    input_variables=["things"],
    portal_variables={"format_instructions": format_instructions}
)

In [73]:
final_prompt = prompt.format(things="countries that play cricket in world cup", format_instructions=format_instructions)

In [74]:
final_prompt

'List 3 countries that play cricket in world cup.\nYour response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [75]:
output = llm.invoke(input=final_prompt)

In [76]:
print(output.strip())

Australia, India, Pakistan


In [77]:
things = output_parser.parse(output)

In [78]:
things

['Australia', 'India', 'Pakistan']

In [79]:
things[1]

'India'

In [80]:
from langchain_core.output_parsers import JsonOutputParser
import json

In [81]:
prompt = PromptTemplate(
    template="List 3 countries in {continent} and their capitals",
    input_variables=["continent"]
)

In [82]:
print(llm.invoke(input=prompt.format(continent="Asia")))

Here are 3 countries in Asia along with their capitals:

1. **Japan** - Tokyo
2. **India** - New Delhi
3. **Singapore** - Singapore City


### Turn in json object

In [83]:
output_parser = JsonOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

Return a JSON object.


In [84]:
prompt = PromptTemplate(
    template="List 3 countries in {continent} and their capitals\n{format_instructions}",
    input_variables=["continent"],
    partial_variables={"format_instructions": format_instructions}
)

In [85]:
print(prompt.format(continent="North America", format_instructions=format_instructions))

List 3 countries in North America and their capitals
Return a JSON object.


In [86]:
output = llm.invoke(input=prompt.format(continent="America"))
print(output)

Here are three countries in the Americas with their capitals:

```
{
    "Mexico": {
        "capital": "Mexico City"
    },
    "Brazil": {
        "capital": "Brasília"
    },
    "United States": {
        "capital": "Washington, D.C."
    }
}
```


In [87]:
countries= output_parser.parse(output)

In [88]:
json.dumps(countries)

'{"Mexico": {"capital": "Mexico City"}, "Brazil": {"capital": "Bras\\u00edlia"}, "United States": {"capital": "Washington, D.C."}}'

In [106]:
from typing import List
from langchain_ollama.llms import OllamaLLM
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
import os

In [107]:
model = OllamaLLM(model="llama3.1:latest")

In [108]:
class Ticket(BaseModel):
    date: str = Field(description="show date")
    time: str = Field(description="show time")
    theater: str = Field(description="theater name")
    count: int = Field(description="number of tickets")
    movie: str = Field(description="preferred movie")

In [109]:
parser = PydanticOutputParser(pydantic_object=Ticket)

In [110]:
ticket_template = '''
  Book us a movie ticket for two this Friday at 6:00 PM. 
  Choose any theater, it doesn't matter. Send the confirmation by email.
  Our preferred movie is: {query}
  Format instructions:
  {format_instructions}
'''

In [ ]:
prompt = PromptTemplate(
    template=ticket_template,
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
input = prompt.format_prompt(query="Interstellar")

In [ ]:
print(input.to_string())

In [ ]:
output = model.invoke(input.to_string())

In [ ]:
parsed_response = parser.parse(output)
print(parsed_response)

In [ ]:
type(parsed_response)